## Лабораторна робота №9 
виконала студентка групи ІС-12 мп

Кравчук Олена

Варіант-9

In [5]:
import findspark
findspark.init()

In [6]:
import pyspark
import random

In [9]:
sc = pyspark.SparkContext(appName="bigData")

In [35]:
from pyspark.sql.functions import isnan, col
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder, MinMaxScaler
from pyspark.sql.types import DoubleType, IntegerType
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
import matplotlib.pyplot as plt
import numpy as np

In [13]:
spark = SparkSession\
    .builder\
    .appName('lab9')\
    .master('local')\
    .getOrCreate()

In [68]:
df = spark.read.format('csv').option('header', True).option('inferSchema', True).option('delimiter',';').load("student-por.csv")
df.toPandas().head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13


In [69]:
def getMissingValuesDescription(df):
    return {
        col: df.filter((df[col] == "") | (df[col] == 'NA') | df[col].isNull() | isnan(df[col])).count()
        for col in df.columns
    }

In [70]:
getMissingValuesDescription(df)

{'school': 0,
 'sex': 0,
 'age': 0,
 'address': 0,
 'famsize': 0,
 'Pstatus': 0,
 'Medu': 0,
 'Fedu': 0,
 'Mjob': 0,
 'Fjob': 0,
 'reason': 0,
 'guardian': 0,
 'traveltime': 0,
 'studytime': 0,
 'failures': 0,
 'schoolsup': 0,
 'famsup': 0,
 'paid': 0,
 'activities': 0,
 'nursery': 0,
 'higher': 0,
 'internet': 0,
 'romantic': 0,
 'famrel': 0,
 'freetime': 0,
 'goout': 0,
 'Dalc': 0,
 'Walc': 0,
 'health': 0,
 'absences': 0,
 'G1': 0,
 'G2': 0,
 'G3': 0}

In [71]:
def getNumericalColumns(df):
    return [
        item[0] for item in df.dtypes 
        if item[1].startswith('int')
    ]

In [72]:
def getCategoricalColumns(df):
    return [
        item[0] for item in df.dtypes 
        if item[1].startswith('string')
    ]

In [73]:
def convertToIntValue(df, columnName):
    df = df.withColumn(columnName, df[columnName].cast(IntegerType()))
    return df

In [74]:
numericalColumns = getNumericalColumns(df)
for col in numericalColumns:
    df = convertToIntValue(df, col)
df.printSchema()

root
 |-- school: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- address: string (nullable = true)
 |-- famsize: string (nullable = true)
 |-- Pstatus: string (nullable = true)
 |-- Medu: integer (nullable = true)
 |-- Fedu: integer (nullable = true)
 |-- Mjob: string (nullable = true)
 |-- Fjob: string (nullable = true)
 |-- reason: string (nullable = true)
 |-- guardian: string (nullable = true)
 |-- traveltime: integer (nullable = true)
 |-- studytime: integer (nullable = true)
 |-- failures: integer (nullable = true)
 |-- schoolsup: string (nullable = true)
 |-- famsup: string (nullable = true)
 |-- paid: string (nullable = true)
 |-- activities: string (nullable = true)
 |-- nursery: string (nullable = true)
 |-- higher: string (nullable = true)
 |-- internet: string (nullable = true)
 |-- romantic: string (nullable = true)
 |-- famrel: integer (nullable = true)
 |-- freetime: integer (nullable = true)
 |-- goout: integer (null

In [75]:
trainDf, testDf = df.randomSplit([0.8, 0.2])

In [76]:
categoricalColumns = getCategoricalColumns(trainDf)
categoricalColumns

['school',
 'sex',
 'address',
 'famsize',
 'Pstatus',
 'Mjob',
 'Fjob',
 'reason',
 'guardian',
 'schoolsup',
 'famsup',
 'paid',
 'activities',
 'nursery',
 'higher',
 'internet',
 'romantic']

In [77]:
indexers = [StringIndexer(inputCol=column, outputCol= column+"_index") for column in categoricalColumns]
pipeline = Pipeline(stages=indexers).fit(trainDf)
trainIndexerDf = (pipeline
             .transform(trainDf))
testIndexerDf = (pipeline
             .transform(testDf))

In [78]:
trainIndexerDf = trainIndexerDf.drop(*categoricalColumns)
testIndexerDf = testIndexerDf.drop(*categoricalColumns)

In [79]:
trainIndexerDf.toPandas().head()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,reason_index,guardian_index,schoolsup_index,famsup_index,paid_index,activities_index,nursery_index,higher_index,internet_index,romantic_index
0,15,1,1,2,4,0,3,1,2,1,...,2.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,15,1,1,3,1,1,5,5,5,1,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
2,15,3,3,2,3,0,4,2,1,2,...,1.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
3,15,3,4,1,3,0,4,3,2,1,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,15,3,4,2,3,0,4,2,2,2,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [80]:
encoders = [OneHotEncoder(inputCol=col+"_index", outputCol=col+'_vector') for col in categoricalColumns]
pipeline = Pipeline(stages=encoders).fit(trainIndexerDf)
trainIndexerDf = (pipeline
             .transform(trainIndexerDf))
testIndexerDf = (pipeline
             .transform(testIndexerDf))

In [81]:
categoricalColumns = [col+"_index" for col in categoricalColumns]
trainDf = trainIndexerDf.drop(*categoricalColumns)
testDf = testIndexerDf.drop(*categoricalColumns)

In [82]:
trainDf.toPandas().head()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,reason_vector,guardian_vector,schoolsup_vector,famsup_vector,paid_vector,activities_vector,nursery_vector,higher_vector,internet_vector,romantic_vector
0,15,1,1,2,4,0,3,1,2,1,...,"(0.0, 0.0, 1.0)","(1.0, 0.0)",(0.0),(1.0),(0.0),(0.0),(1.0),(1.0),(1.0),(1.0)
1,15,1,1,3,1,1,5,5,5,1,...,"(1.0, 0.0, 0.0)","(1.0, 0.0)",(1.0),(0.0),(1.0),(0.0),(1.0),(1.0),(1.0),(0.0)
2,15,3,3,2,3,0,4,2,1,2,...,"(0.0, 1.0, 0.0)","(0.0, 0.0)",(1.0),(1.0),(0.0),(0.0),(1.0),(1.0),(1.0),(0.0)
3,15,3,4,1,3,0,4,3,2,1,...,"(1.0, 0.0, 0.0)","(1.0, 0.0)",(0.0),(1.0),(1.0),(0.0),(1.0),(1.0),(1.0),(1.0)
4,15,3,4,2,3,0,4,2,2,2,...,"(1.0, 0.0, 0.0)","(0.0, 1.0)",(1.0),(1.0),(1.0),(1.0),(1.0),(1.0),(1.0),(0.0)


In [83]:
feature_col = trainDf.columns
feature_col.remove('G3')
df_assembler = VectorAssembler(inputCols=feature_col, outputCol='features')

In [84]:
rf = RandomForestRegressor(labelCol="G3", featuresCol="features")

In [87]:
# Hyperparameter tuning
paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [int(x) for x in np.linspace(start = 10, stop = 50, num = 3)]) \
    .addGrid(rf.maxDepth, [int(x) for x in np.linspace(start = 5, stop = 25, num = 3)]) \
    .build()

In [88]:
pipeline = Pipeline(stages=[df_assembler, rf])

In [111]:
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator(labelCol='G3', metricName="r2"),
                          numFolds=3)

In [112]:
cvModel = crossval.fit(trainDf)
predictions = cvModel.transform(testDf)

In [113]:
cvModel.avgMetrics

[0.7927481656928026,
 0.81004678646896,
 0.8100483105324036,
 0.8144545647371129,
 0.8231843700945722,
 0.8231901872790203,
 0.8091441017062562,
 0.8218428206423511,
 0.8218541382940927]

In [114]:
# Feature importance
bestPipeline = cvModel.bestModel
bestModel = bestPipeline.stages[1]
importances = bestModel.featureImportances

In [115]:
# Best hyperparameters
print('numTrees - ', bestModel.getNumTrees)
print('maxDepth - ', bestModel.getOrDefault('maxDepth'))

numTrees -  30
maxDepth -  25


In [121]:
predictions.select('G3', 'prediction').toPandas().head()

,G3,prediction
0,11,11.900000
1,12,12.866667
2,10,11.100000
3,11,10.800000
4,15,15.233333
